# Using PyTorch for Multiple Output Linear Regression

In [21]:
import torch
import torch.nn as nn
import numpy as np
from torch.utils.data import Dataset, DataLoader
from torch import optim

# Our Linear Regression Class
class LR(nn.Module):
    def __init__(self, input_size, output_size):
        super(LR, self).__init__()
        self.linear = nn.Linear(input_size, output_size)

    def forward(self, x):
        out = self.linear(x)
        return out

In [11]:
# set the seed
torch.manual_seed(1)
# We are setting the number of output nodes to 2
model = LR(input_size=2, output_size=2)
# Print out the model's parameters
print(list(model.parameters()))

[Parameter containing:
tensor([[ 0.3643, -0.3121],
        [-0.1371,  0.3319]], requires_grad=True), Parameter containing:
tensor([-0.6657,  0.4241], requires_grad=True)]


In [12]:
# Create X as containing two input features
x = torch.tensor([[1.0, 2.5]])
# Make a prediction using our model
yhat = model(x)
# Print yhat, notice how yhat contains a value for each output node
print(yhat)

tensor([[-1.0816,  1.1167]], grad_fn=<AddmmBackward0>)


In [13]:
# Same thing as above, but with multiple x values (multiple inputs)
X = torch.tensor([[1.0, 2.5], [2.3, 0.2]])
yhats = model(X)
print(yhats)

tensor([[-1.0816,  1.1167],
        [ 0.1099,  0.1752]], grad_fn=<AddmmBackward0>)


In [16]:
# Create a Dataset Class for training the model

class Data2D(Dataset):
    def __init__(self):
        # create an empty array (or tensor?) with 20 0's across two dimensions
        self.x = torch.zeros(20, 2)
        # create a range of values between -1 an 1 with a 0.1 change in value for each number
        self.x[:, 0] = torch.arange(-1,1,0.1)
        self.x[:, 1] = torch.arange(-1, 1, 0.1)
        self.w = torch.tensor([[1.0, -1.0],[1.0, -1.0]])
        self.b = torch.tensor([1.0, -1.0])
        self.f = torch.mm(self.x, self.w) + self.b
        self.y = self.f  + 0.1 * torch.randn((self.x.shape[0], 1))
        self.len = self.x.shape[0]

    def __getitem__(self, index):
        return self.x[index], self.y[index]

    def __len__(self):
        return self.len

In [26]:
dataset = Data2D()
criterion = nn.MSELoss()
trainloader = DataLoader(dataset=dataset, batch_size=1)

model = LR(input_size=2, output_size=2)
optimizer = optim.SGD(model.parameters(), lr=0.01)

In [29]:
print(list(model.parameters()))
for epoch in range(100):
    for x,y in trainloader:
        yhat = model(X)
        loss = criterion(yhat, y)
        # reset parameter values to 0, this is just a pytorch thing
        optimizer.zero_grad()
        # perform backwards prop
        loss.backward()
        optimizer.step()

print(list(model.parameters()))

[Parameter containing:
tensor([[ 0.3687,  0.2367],
        [-0.3977, -0.2524]], requires_grad=True), Parameter containing:
tensor([ 0.3764, -0.2413], requires_grad=True)]
[Parameter containing:
tensor([[ 0.3687,  0.2367],
        [-0.3977, -0.2524]], requires_grad=True), Parameter containing:
tensor([ 0.3764, -0.2413], requires_grad=True)]
